In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import yfinance as yf
import MetaTrader5 as mt5 #mt5 to access historcal data
import pandas as pd # for data analysis and calculation of technical indcator
import pandas_ta as ta
import numpy as np
import plotly.io as pio
import plotly.express as px # for data visualization
import plotly.graph_objects as go
import tdclient as TDClient
from datetime import datetime, time ,timezone #to specify the date time range for historical data
from IPython.display import display, Markdown, Latex # to display result in python notebook
from backtest import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

: 

In [ ]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

In [ ]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    df = pd.DataFrame(ohlc)
    df['time'] = pd.to_datetime(df['time'], unit= 's')
    return df

In [ ]:
# 1 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M1
start_datetime = datetime(2025,6,1)
end_datetime = datetime.now()
df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
df

In [ ]:
spread_threshold = 5  # in points
atr_period = 14

# Calculate metrics
#df['spread'] = (df['ask'] - df['bid']) * 10000  # Convert to pips
df['vwap'] = (df['close'] * df['tick_volume']).cumsum() / df['tick_volume'].cumsum()
df['atr'] = df[['high', 'low', 'close']].apply(
    lambda row: max(row['high'] - row['low'],
                    abs(row['high'] - row['close']),
                    abs(row['low'] - row['close'])),
    axis=1
)
df['atr'] = df['atr'].rolling(atr_period).mean()

# Strategy condition
df['volume_percentile'] = df['tick_volume'].rolling(atr_period).apply(
    lambda x: np.percentile(x, 75), raw=True
)
df

In [ ]:
df['high'] = df['high'].shift(1)

In [ ]:
def entring_stoploss_profit(row, df) :
    if (row['spread'] < spread_threshold) & (row['close'] > row['vwap']) &  (row['tick_volume'] > row['volume_percentile']) &(row['close'] > row['high']):
        return "buy"
    #elif :
        #return "sell"
df["signal"] = df.apply(entring_stoploss_profit, axis=1, df = df)

# Tag first signal per day in original merged_df
#df['is_first_signal'] = (df[df["signal"].notna()].groupby(df["time"].dt.date).cumcount() == 0)

# Set signal only on the first row per day, others = None
#df['signal'] = df.apply(lambda row: row['signal'] if row['is_first_signal'] else None, axis=1)
df[['signal', 'time']].dropna()

In [ ]:
pd.options.mode.copy_on_write = True
# create trade logic
def on_bar(data, trades, orders):
    data['stop_loss'] = data['close'] - data['atr']
    data['take_profit'] = data['close'] + 2 * data['atr']
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    account_balance = 10000  # your account in USD
    risk_percent = 0.20  # 2%
    pip_value_per_lot = 10  # on EURUSD, 1 lot = $10 per pip
    risk_amount = account_balance * risk_percent
    # entry signal
    if data['signal'] == 'buy': #and not num_open_trades:
        volume = 100000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell': #and not num_open_trades:
        volume = 100000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'sell')
        
# exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and (data['high'] >= data['take_profit']) | (data['low'] <= data['stop_loss']):
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' :
            orders.close_trade(trade)

In [ ]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(df)
bt.set_on_bar(on_bar)

get_ohlc_history(symbol, timeframe, start_datetime, end_datetime)
bt.run_backtest()

bt.trades


In [ ]:
mm = bt.plot_balance()
display(mm)

In [ ]:
evaluate_backtest(bt.trades)